In [1]:
import pathlib
import sys
import time

sys.path.append("../featurization")
from typing import Dict

import cucim
import cucim.skimage.morphology
import cupy as cp
import cupyx.scipy.ndimage
import numpy
import numpy as np
import pandas as pd
import scipy
import skimage
import tqdm
from data_writer import organize_featurization_data
from granularity import measure_3D_granularity_gpu
from loading_classes import ImageSetLoader, ObjectLoader

In [2]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [3]:
channel_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

In [5]:
object_loader = ObjectLoader(
    image_set_loader.image_set_dict["DNA"],
    image_set_loader.image_set_dict["Cell"],
    "DNA",
    "Cell",
)
object_loader.object_ids
image_set_loader.image_set_dict.keys()

dict_keys(['DNA', 'AGP', 'ER', 'Mito', 'BF', 'Cell', 'Cytoplasm', 'Nuclei', 'Organoid'])

In [6]:
def granularity_feature(length):
    C_GRANULARITY = "GRANULARITY.%s"
    return C_GRANULARITY % (length)


class ObjectRecord:
    def __init__(self, object_loader, object_index):
        self.object_index = object_index
        self.labels = object_loader.objects.copy()
        # select the object
        self.labels[self.labels != object_index] = 0
        self.image = object_loader.image.copy()
        self.image[self.labels != object_index] = 0

        # self.labels[self.labels == object_index] = 1
        # self.labels[~object_index] = 0

        self.nobjects = len(numpy.unique(self.labels))
        if self.nobjects != 0:
            self.range = numpy.arange(1, numpy.max(self.labels) + 1)
            self.current_mean = scipy.ndimage.mean(self.image, self.labels)
            self.start_mean = numpy.maximum(self.current_mean, numpy.finfo(float).eps)


object_record = ObjectRecord(object_loader, 1)

In [7]:
# def measure_3D_granularity_gpu(
#         object_loader: ObjectLoader,
#         radius: int = 20,
#         granular_spectrum_length: int = 16,
#         subsample_size: float = 0.25,
#         image_name: str = "nuclei",
#         ) -> pd.DataFrame:

#     pixels = cp.asarray(object_loader.image)
#     mask = cp.asarray(object_loader.label_image)
#     new_shape = cp.asarray(pixels.shape) * subsample_size
#     k,i,j = (
#         cp.mgrid[0:new_shape[0], 0:new_shape[1], 0:new_shape[2]].astype(cp.float32)
#         / subsample_size
#     )
#     pixels = cupyx.scipy.ndimage.map_coordinates(
#         pixels,
#         cp.stack((k,i,j)),
#         order=1
#         )
#     mask = cupyx.scipy.ndimage.map_coordinates(
#         mask,
#         cp.stack((k, i, j)),
#         ) > 0.9
#     back_shape = new_shape * subsample_size

#     back_shape = new_shape * subsample_size
#     k, i, j = (
#         cp.mgrid[0 : new_shape[0], 0 : new_shape[1], 0 : new_shape[2]].astype(cp.float32)
#         / subsample_size
#     )
#     back_pixels = cupyx.scipy.ndimage.map_coordinates(pixels, cp.stack((k, i, j)), order=1)
#     back_mask = cupyx.scipy.ndimage.map_coordinates(mask.astype(cp.float32), cp.stack((k, i, j))) > 0.9
#     footprint = cucim.skimage.morphology.ball(radius, dtype=bool)
#     # conver the footprint bool to float32
#     # footprint = footprint.astype(cp.int32)

#     back_pixels_mask = cp.zeros_like(back_pixels)
#     back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]
#     back_pixels_mask = cp.asarray(back_pixels_mask)

#     back_pixels = cucim.skimage.morphology.isotropic_erosion(
#         back_pixels_mask,
#         radius=3,
#         spacing=image_set_loader.spacing,
#         )

#     back_pixels_mask = cp.zeros_like(back_pixels)
#     back_pixels_mask[back_mask == True] = back_pixels[back_mask == True]

#     back_pixels = cucim.skimage.morphology.isotropic_dilation(
#         back_pixels_mask,
#         radius=3,
#         spacing=image_set_loader.spacing,
#         )
#     k, i, j = cp.mgrid[0 : new_shape[0], 0 : new_shape[1], 0 : new_shape[2]].astype(
#         cp.float32
#     )

#     k *= (back_shape[0] - 1) / (new_shape[0] - 1)
#     i *= (back_shape[1] - 1) / (new_shape[1] - 1)
#     j *= (back_shape[2] - 1) / (new_shape[2] - 1)

#     back_pixels = cupyx.scipy.ndimage.map_coordinates(back_pixels, cp.stack((k, i, j)), order=1)
#     pixels -= back_pixels
#     pixels[pixels < 0] = 0

#     startmean = cp.mean(pixels[mask])
#     ero = pixels.copy()

#     # Mask the test image so that masked pixels will have no effect
#     # during reconstruction
#     ero[~mask] = 0
#     currentmean = startmean
#     startmean = max(startmean, cp.finfo(float).eps)
#     footprint = cucim.skimage.morphology.ball(1, dtype=bool)
#     statistics = [image_name]
#     feature_measurments = {}
#     objects_records = [ObjectRecord(object_loader=object_loader, object_index=object_id) for object_id in object_loader.object_ids]
#     object_measurements = {"object_id": [], "feature": [], "value": []}
#     for i in tqdm.tqdm(range(1, granular_spectrum_length + 1)):
#         prevmean = currentmean
#         ero_mask = cp.zeros_like(ero)
#         ero_mask[mask == True] = ero[mask == True]
#         ero = cucim.skimage.morphology.isotropic_erosion(
#             ero_mask,
#             radius=3,
#             spacing=image_set_loader.spacing,
#             )
#         rec = cucim.skimage.morphology.reconstruction(
#             ero,
#             pixels,
#             footprint=footprint,
#         )
#         currentmean = cp.mean(rec[mask])
#         gs = (prevmean - currentmean) * 100 / startmean
#         statistics += ["%.2f" % gs]
#         feature = granularity_feature(i)
#         feature_measurments[feature] = gs
#         # Restore the reconstructed image to the shape of the
#         # original image so we can match against object labels
#         orig_shape = object_loader.image.shape
#         k, i, j = cp.mgrid[
#             0 : orig_shape[0], 0 : orig_shape[1], 0 : orig_shape[2]
#         ].astype(cp.float32)
#         k *= (new_shape[0] - 1) / (orig_shape[0] - 1)
#         i *= (new_shape[1] - 1) / (orig_shape[1] - 1)
#         j *= (new_shape[2] - 1) / (orig_shape[2] - 1)

#         rec = cupyx.scipy.ndimage.map_coordinates(rec, cp.stack((k, i, j)), order=1)
#         for object_record in objects_records:
#             if object_record.nobjects > 0:
#                 labels = cp.asarray(object_record.labels)
#                 new_mean = cp.mean(rec - labels)
#                 gss = (
#                     (object_record.current_mean - new_mean)
#                     * 100
#                     / object_record.start_mean
#                 )
#                 object_record.current_mean = new_mean

#             else:
#                 gss = cp.zeros((0,))
#             object_measurements["object_id"].append(object_record.object_index)
#             object_measurements["feature"].append(feature)
#             object_measurements["value"].append(gss.get())
#     df = pd.DataFrame(object_measurements)
#     # get the mean of each value in the array
#     # melt the dataframe to wide format
#     df = df.pivot_table(index=["object_id"], columns=["feature"], values=["value"])
#     df.columns = df.columns.droplevel()
#     df = df.reset_index()
#     df.head()

In [8]:
start_time = time.time()

In [10]:
object_measurements = measure_3D_granularity_gpu(
    object_loader=object_loader,
    image_set_loader=image_set_loader,
    radius=20,
    granular_spectrum_length=16,
    subsample_size=0.25,
    image_name=object_loader.channel,
)

KeyboardInterrupt: 

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s minutes ---" % ((time.time() - start_time) / 60))
print("--- %s hours ---" % ((time.time() - start_time) / 3600))

--- 116.5708920955658 seconds ---
--- 1.9428490002950032 minutes ---
--- 0.03238082448641459 hours ---


In [ ]:
object_measurements

In [ ]:
df = pd.DataFrame(object_measurements)
# get the mean of each value in the array
# melt the dataframe to wide format
# df = df.pivot_table(index=["object_id"], columns=["feature"], values=["value"])
# df.columns = df.columns.droplevel()
# df = df.reset_index()
df.head()

""
